<a href="https://colab.research.google.com/github/Muhammadshahidusman/30DayMapChallenge/blob/main/Day_17_Using_python_for_the_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
# ==========================
# Kaghan Valley – Responsive Dashboard with Boundary Polygon
# Mobile-friendly with hollow polygon boundary
# ==========================

!pip install folium geopandas pandas xyzservices

import os
import zipfile
import pandas as pd
import geopandas as gpd
import folium
import xyzservices.providers as xyz

from folium.plugins import BeautifyIcon, FloatImage, Fullscreen, MousePosition
from branca.element import Template, MacroElement, Element

# --------------------------
# 1. Load shapefiles (Points + Boundary)
# --------------------------

# Extract and load POINTS
with zipfile.ZipFile('KDA_Points.zip', 'r') as zip_ref:
    zip_ref.extractall('shapefile_folder')

shp_point = 'shapefile_folder/KDA_PointData.shp'
gdf_points = gpd.read_file(shp_point)
gdf_points = gdf_points.to_crs(epsg=4326)

# Extract and load BOUNDARY POLYGON
with zipfile.ZipFile('KDA_Boundary.zip', 'r') as zip_ref:
    zip_ref.extractall('shapefile_folder')

# Find the boundary shapefile (adjust filename if needed)
boundary_files = [f for f in os.listdir('shapefile_folder') if f.endswith('.shp') and 'boundary' in f.lower()]
if boundary_files:
    boundary_path = f'shapefile_folder/{boundary_files[0]}'
else:
    # If filename is different, specify exact name:
    boundary_path = 'shapefile_folder/KDA_Boundary.shp'  # adjust this name

gdf_boundary = gpd.read_file(boundary_path)
gdf_boundary = gdf_boundary.to_crs(epsg=4326)

print("Boundary loaded:", len(gdf_boundary), "polygon(s)")

# --------------------------
# 2. Categorize features
# --------------------------

def categorize(name):
    if pd.isna(name):
        return 'Other'
    n = str(name).lower()
    if 'bridge' in n:
        return 'Bridge'
    elif 'ls' in n:
        return 'Landslide Area'
    elif 'guardrails' in n or 'quadrails' in n:
        return 'Guardrails'
    elif 'drainage' in n:
        return 'Drainage Issue'
    elif 'rehabilitation' in n or 'maintaince' in n:
        return 'Road Rehabilitation'
    elif any(x in n for x in ['tourist', 'zip line', 'track', 'valley', 'meadows', 'glacier']):
        return 'Tourist Feature'
    elif 'rs' in n:
        return 'Retaining Structure'
    else:
        return 'Other'

gdf_points['Category'] = gdf_points['Name'].apply(categorize)

print("Number of points:", len(gdf_points))
print(gdf_points[['Name', 'Category']].head())

# --------------------------
# 3. Create base map
# --------------------------

# Center on boundary centroid
center_lat = gdf_boundary.geometry.centroid.y.mean()
center_lon = gdf_boundary.geometry.centroid.x.mean()

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=12,
    tiles=None,
    control_scale=True
)

# Multiple base maps
folium.TileLayer(xyz.OpenStreetMap.Mapnik, name='Street').add_to(m)
folium.TileLayer(xyz.CartoDB.Positron, name='Light').add_to(m)
folium.TileLayer(xyz.CartoDB.DarkMatter, name='Dark').add_to(m)
folium.TileLayer(xyz.CartoDB.Voyager, name='Voyager').add_to(m)
folium.TileLayer(xyz.Esri.WorldImagery, name='Satellite').add_to(m)

# --------------------------
# 4. Add KDA Boundary (HOLLOW with beautiful border)
# --------------------------

# Style function for hollow polygon: no fill, thick colored border
def boundary_style(feature):
    return {
        'fillColor': 'none',        # No fill - transparent inside
        'fillOpacity': 0,            # Completely transparent
        'color': '#FF6B35',          # Bold orange-red border
        'weight': 3,                 # Thick border line
        'opacity': 0.9,              # Border opacity
        'dashArray': '10, 5'         # Dashed line (optional, remove for solid)
    }

# Add boundary to map
boundary_layer = folium.FeatureGroup(name='KDA Boundary', show=True)

folium.GeoJson(
    gdf_boundary,
    style_function=boundary_style,
    tooltip=folium.GeoJsonTooltip(fields=[], aliases=[], labels=False)
).add_to(boundary_layer)

boundary_layer.add_to(m)

# --------------------------
# 5. Category styles + markers
# --------------------------

category_styles = {
    'Bridge': {
        'icon': 'road',
        'border_color': '#8B0000',
        'background_color': 'white',
        'text_color': '#8B0000',
        'legend_color': '#8B0000'
    },
    'Landslide Area': {
        'icon': 'exclamation-triangle',
        'border_color': '#FF8C00',
        'background_color': 'white',
        'text_color': '#FF8C00',
        'legend_color': '#FF8C00'
    },
    'Guardrails': {
        'icon': 'shield',
        'border_color': '#000000',
        'background_color': 'white',
        'text_color': '#000000',
        'legend_color': '#000000'
    },
    'Drainage Issue': {
        'icon': 'tint',
        'border_color': '#0066CC',
        'background_color': 'white',
        'text_color': '#0066CC',
        'legend_color': '#0066CC'
    },
    'Road Rehabilitation': {
        'icon': 'wrench',
        'border_color': '#DC143C',
        'background_color': 'white',
        'text_color': '#DC143C',
        'legend_color': '#DC143C'
    },
    'Tourist Feature': {
        'icon': 'camera',
        'border_color': '#228B22',
        'background_color': 'white',
        'text_color': '#228B22',
        'legend_color': '#228B22'
    },
    'Retaining Structure': {
        'icon': 'square',
        'border_color': '#808080',
        'background_color': 'white',
        'text_color': '#808080',
        'legend_color': '#808080'
    },
    'Other': {
        'icon': 'info-sign',
        'border_color': '#9932CC',
        'background_color': 'white',
        'text_color': '#9932CC',
        'legend_color': '#9932CC'
    }
}

unique_categories = gdf_points['Category'].unique()

for cat in unique_categories:
    fg = folium.FeatureGroup(name=f"Category: {cat}", show=True)
    subset = gdf_points[gdf_points['Category'] == cat]
    style = category_styles.get(cat, category_styles['Other'])

    for _, row in subset.iterrows():
        icon = BeautifyIcon(
            icon=style['icon'],
            icon_shape='marker',
            border_color=style['border_color'],
            background_color=style['background_color'],
            text_color=style['text_color'],
            spin=False
        )
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            popup=f"<b>{row['Name']}</b><br>Category: {row['Category']}",
            icon=icon
        ).add_to(fg)
    fg.add_to(m)

# --------------------------
# 6. Responsive styled title
# --------------------------

title_html = '''
<div class="map-title" style="position: fixed;
            top: 10px;
            left: 50%;
            transform: translateX(-50%);
            max-width: 90%;
            width: 600px;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            border-radius: 10px;
            box-shadow: 0 4px 15px rgba(0,0,0,0.3);
            z-index: 9999;
            text-align: center;
            padding: 15px 10px;
            color: white;
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;">
    <h2 style="margin: 0; font-size: 20px; font-weight: 600; letter-spacing: 1px;">
        🗺️ Kaghan Valley – KDA Road Safety & Tourism Inventory
    </h2>
    <p style="margin: 5px 0 0 0; font-size: 11px; opacity: 0.95;">
        Interactive Dashboard | #30DayMapChallenge Day 17
    </p>
</div>

<style>
@media (max-width: 768px) {
    .map-title h2 { font-size: 16px !important; }
    .map-title p { font-size: 10px !important; }
    .map-title { width: 95% !important; padding: 10px 8px !important; top: 5px !important; }
}
</style>
'''
m.get_root().html.add_child(Element(title_html))

# --------------------------
# 7. Custom legend (LOWER-RIGHT, HIDDEN ON MOBILE)
# --------------------------

legend_items = ""
for cat in sorted(unique_categories):
    style = category_styles.get(cat, category_styles['Other'])
    color = style['legend_color']
    legend_items += f'<li><span style="background:{color};"></span>{cat}</li>\n'

# Add boundary to legend
legend_items += f'<li><span style="background:none; border: 3px dashed #FF6B35; height:14px; width:14px;"></span>KDA Boundary</li>\n'

legend_template = f"""
{{% macro html(this, kwargs) %}}

<!doctype html>
<html lang="en">
<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <link rel="stylesheet" href="//code.jquery.com/ui/1.12.1/themes/base/jquery-ui.css">
  <script src="https://code.jquery.com/jquery-1.12.4.js"></script>
  <script src="https://code.jquery.com/ui/1.12.1/jquery-ui.js"></script>

  <script>
  $( function() {{
    $( "#maplegend" ).draggable({{
                    start: function (event, ui) {{
                        $(this).css({{
                            right: "auto",
                            top: "auto",
                            bottom: "auto"
                        }});
                    }}
                }});
}});
  </script>
</head>
<body>

<div id='maplegend' class='maplegend'
    style='position: fixed;
           z-index: 9999;
           border: 2px solid #999;
           background: linear-gradient(to bottom, #ffffff 0%, #f4f4f9 100%);
           border-radius: 8px;
           padding: 12px;
           font-size: 13px;
           right: 15px;
           bottom: 60px;
           box-shadow: 0 4px 12px rgba(0,0,0,0.2);
           font-family: "Segoe UI", Tahoma, Geneva, Verdana, sans-serif;
           max-width: 200px;'>

<div class='legend-title' style='font-size: 14px; font-weight: 600; margin-bottom: 8px; color: #333;'>
    📍 Map Legend
</div>
<div class='legend-scale'>
  <ul class='legend-labels'>
    {legend_items}
  </ul>
</div>
<div class='legend-source' style='margin-top: 6px; padding-top: 6px; border-top: 1px solid #ddd; font-size: 9px; color: #777;'>
    Drag to move
</div>
</div>

</body>
</html>

<style type='text/css'>
  .maplegend .legend-scale ul {{
    margin: 0;
    padding: 0;
    list-style: none;
  }}
  .maplegend .legend-scale ul li {{
    font-size: 11px;
    list-style: none;
    margin-left: 0;
    line-height: 20px;
    margin-bottom: 4px;
  }}
  .maplegend ul.legend-labels li span {{
    display: inline-block;
    height: 14px;
    width: 14px;
    margin-right: 6px;
    border: 1px solid #999;
    border-radius: 2px;
    vertical-align: middle;
  }}

  /* HIDE LEGEND ON MOBILE (screens < 768px) */
  @media (max-width: 768px) {{
    .maplegend {{
      display: none !important;
    }}
  }}
</style>
{{% endmacro %}}
"""

macro = MacroElement()
macro._template = Template(legend_template)
m.get_root().add_child(macro)

# --------------------------
# 8. North arrow (UPPER-RIGHT, smaller, responsive)
# --------------------------

north_arrow_html = '''
<div class="north-arrow" style="position: fixed;
                                 top: 110px;
                                 right: 15px;
                                 z-index: 9999;
                                 background: white;
                                 border: 2px solid #999;
                                 border-radius: 50%;
                                 padding: 5px;
                                 box-shadow: 0 2px 8px rgba(0,0,0,0.2);">
    <img src="https://raw.githubusercontent.com/ocefpaf/secoora_assets_map/a250729bbcf2ddd12f46912d36c33f7539131bec/secoora_icons/rose.png"
         style="width: 60px; height: 60px; display: block;">
</div>

<style>
@media (max-width: 768px) {
    .north-arrow {
        top: 80px !important;
        right: 10px !important;
    }
    .north-arrow img {
        width: 45px !important;
        height: 45px !important;
    }
}
</style>
'''
m.get_root().html.add_child(Element(north_arrow_html))

# --------------------------
# 9. Fullscreen & mouse position
# --------------------------

Fullscreen(position='topleft').add_to(m)

MousePosition(
    position="bottomright",
    prefix="Coords:",
    separator=" | "
).add_to(m)

# --------------------------
# 10. Layer control (LOWER-LEFT, HIDDEN ON MOBILE)
# --------------------------

layer_control_css = '''
<style>
    .leaflet-control-layers {
        position: fixed !important;
        left: 15px !important;
        bottom: 60px !important;
        top: auto !important;
        right: auto !important;
    }

    /* HIDE LAYER CONTROL ON MOBILE (screens < 768px) */
    @media (max-width: 768px) {
        .leaflet-control-layers {
            display: none !important;
        }
    }
</style>
'''
m.get_root().html.add_child(Element(layer_control_css))

folium.LayerControl(collapsed=False, position='topleft').add_to(m)

# --------------------------
# 11. Save
# --------------------------

output_name = 'kaghan_final_dashboard.html'
m.save(output_name)
print("✅ Final map with boundary saved to:", os.path.join(os.getcwd(), output_name))

m


Boundary loaded: 1 polygon(s)
Number of points: 114
                          Name         Category
0  2 span concrete road bridge           Bridge
1         Abshar tourist point  Tourist Feature
2     Batakundi starting point            Other
3                       Bridge           Bridge
4                       Bridge           Bridge


/tmp/ipython-input-1617438468.py:82: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lat = gdf_boundary.geometry.centroid.y.mean()
/tmp/ipython-input-1617438468.py:83: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center_lon = gdf_boundary.geometry.centroid.x.mean()


✅ Final map with boundary saved to: /content/kaghan_final_dashboard.html
